In [9]:
import os
import re
import pandas as pd
import numpy as np
from collections import Counter

In [10]:
df_cancer_related = pd.read_csv('/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/code/outputs/databases/cancer_related_papers_w_text.csv')
print(len(df_cancer_related['title'].unique()))

df_cancer_related

263


,title,header_no,header_title,text,volume
0,Anatomy-Driven Pathology Detection on Chest X-...,1.0,Introduction,chest radiographs (chest x-rays) represent the...,1
1,Anatomy-Driven Pathology Detection on Chest X-...,2.0,Related Work,weakly supervised pathology detection. due to ...,1
2,Anatomy-Driven Pathology Detection on Chest X-...,3.1,Model,figure 1 provides an overview of our method. g...,1
3,Anatomy-Driven Pathology Detection on Chest X-...,3.2,Inference,"during inference, the trained model predicts a...",1
4,Anatomy-Driven Pathology Detection on Chest X-...,3.3,Training,the anatomical region detector is trained usin...,1
...,...,...,...,...,...
2512,LightNeuS: Neural Surface Reconstruction in En...,3.1,Using Illumination Decline as a Depth Cue,the neus formulation of sect. 2 assumes distan...,10
2513,LightNeuS: Neural Surface Reconstruction in En...,3.2,Endoscope Photometric Model,"apart from illumination decline, there are sev...",10
2514,LightNeuS: Neural Surface Reconstruction in En...,4.0,Experiments,we validate our method on the c3vd dataset [4]...,10
2515,LightNeuS: Neural Surface Reconstruction in En...,5.0,Conclusion,we have presented a method for 3d dense multi-...,10


In [26]:
unique_header_titles = df_cancer_related['header_title'].unique()

In [31]:
unique_header_titles_df = pd.DataFrame(unique_header_titles, columns=['header_titles'])
unique_header_titles_df.to_csv('unique_header_titles.csv')

### Extract keyword-related sentences from selected papers
***

#### Demographics only

##### Long keyword lists

In [32]:
keywords_demographics_long = [
    'age', 'gender', 'sex', 'women', 'woman', 'female', 'male',
    'geolocation', 'geographical', 'geographic', 'country', 'countries', 'city', 'cities', 
    'hospital', 'hospitals', 'clinic', 'clinics', 'society', 'societies',
    'etnicity', 'etnicities', 'race', 
    'bias', 'biases', 'fair', 'unfair', 'fairness', 'transparency', 'awareness',
    'imbalance', 'imbalanced', 'balance', 'balanced',
    'problem', 'problems', 'issue', 'issues', 'challenge', 'challenges', 
    'difficult', 'difficulty', 'difficulties']


keywords_demographics_short = [
    'age', 'gender', 'sex', 'women', 'woman', 'female', 'male',
    'patient', 'patients', 'etnicity', 'etnicities', 'race', 
    'bias', 'biases','fairness', 'transparency', 
    'demographic', 'demographics']

##### Short keywords list

In [88]:

# List of keywords
keywords_age        = ['age']

keywords_gender     = ['gender', 'sex', 'women', 'woman', 'female', 'male']

keywords_etnicity   = ['etnicity', 'etnicities', 'race']

keywords_geoloc     = ['geolocation', 'geographical', 'geographic', 'country', 'countries', 'city', 'cities', 
                        'hospital', 'hospitals', 'clinic', 'clinics', 'society', 'societies',]

keywords_bias       = ['bias', 'biases','fairness']

#### Dataset and demographics

In [6]:
# List of keywords
keywords_demographics_data = [
    'age', 'gender', 'sex', 'women', 'woman', 'female', 'male',
    'patient', 'patients', 'etnicity', 'etnicities', 'race', 
    'bias', 'biases', 'fairness', 'transparency', 
    'imbalance', 'imbalanced', 'balance', 'balanced',
    'demographics', 'demographic', 'data collection']

#### Dataset only

In [10]:
keywords_dataset_1 = ['data', 'dataset', 'datasets']

keywords_dataset_2 = ['private dataset', 'private datasets',
                       'public dataset', 'public datasets',  
                       'dataset', 'datasets']

***
#### Split the text into words and extract keyword-matches. Group each keyword-match by relatd paper 
***

In [89]:
def wrap_text(text, width=100):
    """
    A simple function to wrap text at a given width.
    """
    if pd.isnull(text):
        return text  # Handle NaN values
    
    wrapped_lines = []
    for paragraph in text.split('\n'):  # Splitting by existing newlines to preserve paragraph breaks
        line = ''
        for word in paragraph.split():
            if len(line) + len(word) + 1 > width:
                wrapped_lines.append(line)
                line = word
            else:
                line += (' ' + word if line else word)
        wrapped_lines.append(line)
    return '\n'.join(wrapped_lines)

### Keywords search only
***

In [90]:
# Split the text into sentences and search for the keywords

def extract_keywords(df, keywords):
    # Search for the whole word in the text
    pattern = r'\b(' + '|'.join(re.escape(keyword) for keyword in keywords) + r')\b'

    # Initialize a dictionary to hold sentences organized by paper title
    sentences_by_paper = {}

    # Loop through each row in the dataframe
    for index, row in df_cancer_related.iterrows():
        # Find all sentences that contain any of the keywords
        sentences = re.findall(pattern, row['text'], flags=re.IGNORECASE | re.DOTALL)
        
        # If there are matching sentences, add them to the dictionary under the paper title
        if sentences:
            paper_title = row['title']
            if paper_title not in sentences_by_paper:
                sentences_by_paper[paper_title] = []
            sentences_by_paper[paper_title].extend(sentences)

    # Sentences_by_paper contains all the sentences that contain keywords, organized by paper title

    # Convert this dictionary into a DataFrame:
    # Create a list of tuples (paper title, sentence)        
    keywords_data = [(title, keyword_sentence) for title, related_group in sentences_by_paper.items() for keyword_sentence in related_group]
    keywords_df = pd.DataFrame(keywords_data, columns=['title', 'keyword']) 

    return keywords_df       


In [81]:
#keywords_df = extract_keywords(df_cancer_related, keywords_demographics_long)
#keywords_df = extract_keywords(df_cancer_related, keywords_demographics_short)
#keywords_df = extract_keywords(df_cancer_related, keywords_demographics_data)
#keywords_df = extract_keywords(df_cancer_related, keywords_dataset_2)
#keywords_df = extract_keywords(df_cancer_related, keywords_bias)

#keywords_df.to_csv("/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/code/outputs/data/bias_related_keywords.csv", index=False)
#keywords_df.to_csv("/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/code/outputs/data/cancer_related_keywords.csv", index=False)
#keywords_df.to_csv("/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/code/outputs/data/demographics_data_related_keywords_.csv", index=False)

***
### Sentence search only by list of keywords
***

In [ ]:
# Pseudo code
# regex to split the text into sentences. A sentence is defined as a sequence of characters that ends with a period, question mark, or exclamation mark.
# iterate through the sentences to find those with a keyword from the list of keywords. 
# for each match
    # option 1) concatentinate the previous and next sentences to the sentence with the keyword (if they haven't been added already)
    # option 2) extract sentence with keyword only
# keep track of the sentences already added for each paper title.
# if no matches are found for a paper title, add 'none'.

***



In [91]:
# EXTRACT SENTS WITH KEYWORDS    
# Option 2) Storing keyword sentence only 

def extract_keyword_sentences(df, keywords):
    """
    Extract sentences containing specified keywords from DataFrame and organize by paper title.

    Parameters:
    - df: DataFrame containing the text to search through.
    - keywords: List of keywords to search for in the text.

    Returns:
    - A dictionary with paper titles as keys and lists of sentences containing the keywords as values.
    """

    # Compile the regular expression for matching sentences containing the keywords
    keyword_pattern = re.compile(r'\b(?:' + '|'.join(keywords) + r')\b', flags=re.IGNORECASE)

    # Initialize a dictionary to hold sentences organized by paper title
    sentences_by_paper = {}

    # Loop through each paper title in the DataFrame
    for title in df['title'].unique():
        # Get the full text for the current title
        text = ' '.join(df[df['title'] == title]['text'])
        # Split the text into sentences
        sentences = re.split(r'(?<=[.?!])\s+', text)

        # List to store sentences that contain the keyword
        keyword_sentences_buffer = []

        # Iterate through sentences to find and store sentences that contain the keyword
        for sentence in sentences:
            if keyword_pattern.search(sentence):
                # Add only the sentence with the keyword to the buffer
                keyword_sentences_buffer.append(sentence)

        # Add the sentences to the dictionary, use 'none' if there are no matches
        sentences_by_paper[title] = keyword_sentences_buffer if keyword_sentences_buffer else ['none']
    
    extracted_data = [(title, keyword_sentence) for title, related_group in sentences_by_paper.items() for keyword_sentence in related_group]
    extracted_df = pd.DataFrame(extracted_data, columns=['title', 'extracted_keyword_sent'])
    
    # Wrap title and the extracted sentences to a maximum width of n-characters for better readability
    extracted_df['extracted_keyword_sent'] = extracted_df['extracted_keyword_sent'].apply(wrap_text, width=80)

    return extracted_df

In [105]:

# List of keywords
keywords_age        = ['age']

keywords_gender     = ['gender', 'sex', 'women', 'woman', 'female', 'male']

keywords_etnicity   = ['etnicity', 'etnicities', 'race', 'white patients', 'black patients']

keywords_geoloc     = ['geolocation', 'geographical', 'geographic', 'country', 'countries', 'city', 'cities', 
                        'hospital', 'hospitals', 'clinic', 'clinics', 'society', 'societies',]

keywords_bias       = ['bias', 'biases','fairness']

keywords_data       = ['dataset', 'datasets', 'data collection', 'data collections']

keywords_collected  = ['collected']

In [106]:
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_age).to_csv('age_relanted_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_gender).to_csv('gender_related_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_etnicity).to_csv('etnicity_relanted_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_geoloc).to_csv('geoloc_relanted_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_bias).to_csv('bias_related_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_data).to_csv('data_related_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_collected).to_csv('datacollected_related_sentences.csv')


In [ ]:
# FIND INFO ABOUT DEMOGRAPHIC INFORMATION
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_demographics_long).to_csv('extracted_sentences.csv')

In [ ]:
# FIND INFO ABOUT DEMOGRAPHIC INFORMATION KEYWORD LIST = KEYWORDS_DEMOGRAPHICS_DATA
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_demographics_data).to_csv('demographics_data_related_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_demographics_short)#.to_csv('demographics_data_related_sentences.csv')

# FIND INFO ABOUT DATASETS    
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_dataset).to_csv('/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/code/outputs/data/dataset_extracted_sentences.csv')
#extracted_data = extract_keyword_sentences(df_cancer_related, keywords_dataset_2).to_csv('/Users/yasminsarkhosh/Documents/GitHub/machine-learning-bsc-thesis-2024/code/outputs/data/dataset_private_public_extracted_sentences.csv')

***
***
***

***
***
***